In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census
from pprint import pprint
# import gmaps

# Google API Key
from config import gkey




In [28]:
# geocoordinates
# uses the census api to generate 20 different citties baised on the peramiters that the cities have to be a certain size
# use those 20 citities to run through the df and from there build the analysis
target_coordinates = "33.7537, -84.3863"
# target_coordinates = "43.6187102, -116.2146068"
# target_search = "mexican"
target_search = "restaurant"
# meters is the units
target_radius = 30000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# frame_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=input=location=33.753746%2C+-84.386330&keyword=mexican&radius=1000&type=restaurant&key=API_KEY_HERE"  

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [29]:
# print the response url, avoid doing for public github repos in order to avoid exposing key
# print(response.url)

In [30]:
# convert response to json
places_data = response.json()
    
# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))
# places_data_json = pd.read_json('base_url')


{
    "html_attributions": [],
    "next_page_token": "Aap_uEDmH_CgXbpbxZF5lXKfjIR5j02WqCN8r2lHveqIgTYbl4jW_weQwr_QgkLgpK5MqS5ntRupbXGpGR-o5uOB_eNQ67g-dx1rSqtYgTJFAE5naRXQQed6UzH2PhE4Mup_AmBGUPS5EwMc1JGoFZKyuAY7FBvH_63YnC6xc-rKTSPf4GWaiVKUi9hyxUmVMGILXL5A5mHZYmvvRxuEYuD-uBddsaF7AN9ruIw-FDSIHsQukiuQ4-z7lvZIP6eOkFZtWBGZqj1dEDp2ygCSv5PAA-BNd11DBGd8uabO5vNjOuJEKsglMtTZQW6XselSVYDnkOmF1yh1c9aA_UBkx6tfwbtYBxyd4uPvUazlPTIyU9SaG_HpDweLofj_fYSSakb4TEN-BzjaMkSi9ZAN_QdX3MrHXZEmqjI3A9MmcIjc81676irI9xJqhdfo3cixbUkb3tUs2cwwvc3x",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 33.9019799,
                    "lng": -84.45524859999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.90341642989272,
                        "lng": -84.45398067010727
                    },
                    "southwest": {
                      

In [31]:
# # trying to expand past the first 20 search options
# places_data["next_page_token"]
# params['page_token'] = places_data['next_page_token']
# response = requests.get(base_url, params=params)
# convert response to json
# places_data = response.json()
    
# # Print the json (pretty printed)
# print(json.dumps(places_data, indent=4, sort_keys=True))

In [32]:
len(places_data["results"])

20

In [33]:
# Built the empty columns
name = []
rating = []
user_ratings_total = []
latitude = []
longitude = []
business_status = [] 
price_level = []
vicinity = []

for r in range(len(places_data["results"])):
    #try/except block
    try:
        business_status.append(places_data["results"][r]["business_status"])
        latitude.append(places_data["results"][r]["geometry"]["location"]["lat"])
        longitude.append(places_data["results"][r]["geometry"]["location"]["lng"])
        name.append(places_data["results"][r]["name"])
#         how do we get this to print the true number and not just a true/false statement 
        if places_data["results"][r]["rating"]>= 4.0:
            rating.append(places_data["results"][r]["rating"])
        
        user_ratings_total.append(places_data["results"][r]["user_ratings_total"]>= 500)
        vicinity.append(places_data["results"][r]["vicinity"])
        price_level.append(places_data["results"][r]["price_level"])
    except Exception:
    #if there is a 0 the restaurant has no price level ascoiated with it.
        print("Restaurant has no price level. Skipping...")
   
        price_level.append(0)

   
review_dict = {
#     "Restaurant":name,
#     "Rating":rating,
#     "Business Status":business_status,
#     "user_ratings_total":user_ratings_total,
#     "latitude":latitude,
#     "longitude":longitude,
#     "vicinity":vicinity,
#     "price_level":price_level
    
    "Restaurant":name,
    "Rating":rating,
    "price_level":price_level,
    "user_ratings_total":user_ratings_total,
    "latitude":latitude,
    "longitude":longitude,
    "vicinity":vicinity,
    "Business Status":business_status
    
    
} 
# establish dataframe
restarunt_dataframe = pd.DataFrame(review_dict)
restarunt_dataframe

Restaurant has no price level. Skipping...
Restaurant has no price level. Skipping...
Restaurant has no price level. Skipping...
Restaurant has no price level. Skipping...


,Restaurant,Rating,price_level,user_ratings_total,latitude,longitude,vicinity,Business Status
0,Mambo Italian Street Food,4.0,2,True,33.901980,-84.455249,"2022 Powers Ferry Rd #240, Atlanta",OPERATIONAL
1,Padella Italian Atlanta,4.4,0,False,33.905983,-84.466315,"2999 Windy Hill Rd SE, Marietta",OPERATIONAL
2,Zafron Restaurant,4.3,2,True,33.925691,-84.380051,"236 Johnson Ferry Rd, Sandy Springs",OPERATIONAL
3,Mojave Restaurant,4.3,0,True,33.903085,-84.435175,"6450 Powers Ferry Rd NW, Sandy Springs",OPERATIONAL
4,Mezza Luna Pasta and Seafood,4.7,2,True,33.882835,-84.504890,"1669 Spring Rd SE, Smyrna",OPERATIONAL
5,The Select Restaurant + Bar,4.4,0,True,33.924988,-84.381124,"6405 Blue Stone Rd Suite 200, Sandy Springs",OPERATIONAL
6,Sinbad Mediterranean Grill- MARIETTA,4.4,1,False,33.904364,-84.466210,"3000 Windy Hill Rd SE #164, Marietta",OPERATIONAL
7,Restaurant 10,4.2,2,True,33.754962,-84.403282,"10 Northside Dr NW, Atlanta",OPERATIONAL
8,Pappadeaux Seafood Kitchen,4.5,3,True,33.903148,-84.470340,"2830 Windy Hill Rd SE, Marietta",OPERATIONAL
9,Canoe,4.7,3,True,33.859740,-84.455173,"4199 Paces Ferry Rd SE, Atlanta",OPERATIONAL


In [8]:
# Print the name and address of the first restaurant that appears
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Mambo Italian Street Food
2022 Powers Ferry Rd #240, Atlanta


In [12]:
target_coordinates = "43.6187102, -116.2146068"
# target_search = "mexican"
target_search = "restaurant"
# meters is the units
target_radius = 30000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# frame_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=input=location=33.753746%2C+-84.386330&keyword=mexican&radius=1000&type=restaurant&key=API_KEY_HERE"  

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [13]:
# convert response to json
places_data = response.json()
    
# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "Aap_uECVeBcqqGph-GxdvQpyNL_Z7XkeGwjwINti9V4dlNVwLPWpOBe-h2hJ1_FymLywXVocxNyECinrwvsyF_tHxB6qIsyashrq8FjruEaFMRDMHWikvBw_8P_AoMD_TpCAzondyPtdTk6QLFthLuxeOQdTFhHdkmBdFk4eYel_FhMMNqfvNjnGHIjUxk7qWlyBLidmwqFzYBrX0_wCavdgeEfWuKrulstutuwXR-UAf38ngbEu461-lyVWvVU0BD2DwrXkxGkal93nl24oNFRRJkW1OWHDhwW6u_2R97JKyLfM2H2KF3064o4PZ0ZsPcR04bpYphP7dKnLrT2cQYpzRG9GhLymZQZ37ZmDgrEBFplVg3VmCcdjo1Ifpmi_LeBmIzOSqvQw8uWXveXr1jhFRwaenbOy4g5-FsuZs8l5O_Zc1oy9N5RrQQadq_PXHb_Bdj5HMNz6yhkc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 43.6137094,
                    "lng": -116.2038796
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.61509327989272,
                        "lng": -116.2026776201073
                    },
                    "southwest": {
                        "lat

In [14]:
# convert response to json
places_data = response.json()
    
# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "Aap_uECVeBcqqGph-GxdvQpyNL_Z7XkeGwjwINti9V4dlNVwLPWpOBe-h2hJ1_FymLywXVocxNyECinrwvsyF_tHxB6qIsyashrq8FjruEaFMRDMHWikvBw_8P_AoMD_TpCAzondyPtdTk6QLFthLuxeOQdTFhHdkmBdFk4eYel_FhMMNqfvNjnGHIjUxk7qWlyBLidmwqFzYBrX0_wCavdgeEfWuKrulstutuwXR-UAf38ngbEu461-lyVWvVU0BD2DwrXkxGkal93nl24oNFRRJkW1OWHDhwW6u_2R97JKyLfM2H2KF3064o4PZ0ZsPcR04bpYphP7dKnLrT2cQYpzRG9GhLymZQZ37ZmDgrEBFplVg3VmCcdjo1Ifpmi_LeBmIzOSqvQw8uWXveXr1jhFRwaenbOy4g5-FsuZs8l5O_Zc1oy9N5RrQQadq_PXHb_Bdj5HMNz6yhkc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 43.6137094,
                    "lng": -116.2038796
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.61509327989272,
                        "lng": -116.2026776201073
                    },
                    "southwest": {
                        "lat

In [15]:
name = []
rating = []
user_ratings_total = []
latitude = []
longitude = []
business_status = [] 
price_level = []
vicinity = []

for r in range(len(places_data["results"])):
    #try/except block
    try:
        business_status.append(places_data["results"][r]["business_status"])
        latitude.append(places_data["results"][r]["geometry"]["location"]["lat"])
        longitude.append(places_data["results"][r]["geometry"]["location"]["lng"])
        name.append(places_data["results"][r]["name"])
        rating.append(places_data["results"][r]["rating"])
        user_ratings_total.append(places_data["results"][r]["user_ratings_total"])
        vicinity.append(places_data["results"][r]["vicinity"])
        price_level.append(places_data["results"][r]["price_level"])
    except Exception:
    #if there is a 0 the restaurant has no price level ascoiated with it.
        print("Restaurant has no price level. Skipping...")
   
        price_level.append(0)

   
review_dict = {
#     "Restaurant":name,
#     "Rating":rating,
#     "Business Status":business_status,
#     "user_ratings_total":user_ratings_total,
#     "latitude":latitude,
#     "longitude":longitude,
#     "vicinity":vicinity,
#     "price_level":price_level
    
    "Restaurant":name,
    "Rating":rating,
    "price_level":price_level,
    "user_ratings_total":user_ratings_total,
    "latitude":latitude,
    "longitude":longitude,
    "vicinity":vicinity,
    "Business Status":business_status
    
    
} 
# establish dataframe
restarunt_dataframe = pd.DataFrame(review_dict)
restarunt_dataframe

Restaurant has no price level. Skipping...
Restaurant has no price level. Skipping...
Restaurant has no price level. Skipping...
Restaurant has no price level. Skipping...
Restaurant has no price level. Skipping...


,Restaurant,Rating,price_level,user_ratings_total,latitude,longitude,vicinity,Business Status
0,Trillium Restaurant,4.5,0,345,43.613709,-116.203880,"245 S Capitol Blvd, Boise",OPERATIONAL
1,Cottonwood Grille,4.4,3,1324,43.610960,-116.210104,"913 W River St, Boise",OPERATIONAL
2,Eddie's Restaurant,4.4,1,1446,43.589804,-116.269658,"7067 W Overland Rd, Boise",OPERATIONAL
3,Fork,4.5,2,2894,43.616389,-116.203056,"199 N 8th St, Boise",OPERATIONAL
4,Roghani's Restaurant,4.8,0,98,43.693291,-116.399654,"176 S Rosebud Ln, Eagle",OPERATIONAL
5,Epi's A Basque Restaurant,4.8,3,753,43.614122,-116.391952,"1115 N Main St, Meridian",OPERATIONAL
6,Owyhee Tavern - Steak and Seafood Restaurant,4.5,3,905,43.617329,-116.207345,"1109 W Main St, Boise",OPERATIONAL
7,Asiago's,4.3,2,893,43.617222,-116.205950,"1002 W Main St, Boise",OPERATIONAL
8,Barbacoa Grill,4.7,3,3765,43.597606,-116.183604,"276 Bobwhite Ct, Boise",OPERATIONAL
9,Bacquet's French Restaurant,4.6,0,328,43.694027,-116.339498,"1117 E Winding Creek Dr #150, Eagle",OPERATIONAL


In [ ]:
# import requests

# url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=formatted_address%2Cname%2Crating%2Copening_hours%2Cgeometry&key=API_KEY_HERE"

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload)

# print(response.text)

In [ ]:
# Print the name and address of the first restaurant that appears
# print(places_data["results"][0]["name"]),(places_data["results"][0]["vicinity"])
# print(places_data["results"][0]["vicinity"])
# print(places_data["results"][10]["name"])
# print(places_data["results"][10]["vicinity"])


# name = []
# rating = []
# user_ratings_total = []
# latitude = []
# longatiude = []
# business_satus = [] 
# place_id = []








In [ ]:
# df_nested_list = pd.json_normalize(base_url, record_path =['name'])


# df = pd.read_json(base_url)
# df.head()


# build our data frame columns
name = []
rating = []
user_ratings_total = []
lat = []
lng = []
business_status = []
price_level = []

# Loop through the list of cities and perform a request for data on each
# for i, row in places_data.iterrows():
# # for restaurnts in places_data:
#     response = requests.get(base_url, params=params).json()
    
    
    
#     name.append(response["results"][i]["name"])
#     rating.append(response["results"][i]["rating"])
#     user_ratings_total.append(response["results"][i]["user_ratings_total"])
#     lat.append(response["results"][i]["geometry"]["location"]["lat"])
#     lng.append(response["results"][i]["geometry"]["location"]["lng"])
#     business_status.append(response["results"][i]["business_status"])
#     price_level.append(response["results"][i]["price_level"])

#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
print(name)

In [ ]:
# atl_df=pd.DataFrame({"places_data":places_data})
# atl_df["place_id"] = ""
# atl_df["name"] = ""
# atl_df["rating"] = ""
# atl_df["user_ratings_total"] = ""
# atl_df["lat"] = ""
# atl_df["lng"] = ""
# atl_df["business_status"] = ""
# create a data frame from cities, lat, and temp
# weather_dict = {
#     "city": cities,
#     "lat": lat,
#     "temp": temp
# }
# weather_data = pd.DataFrame(weather_dict)
# weather_data.head()



# atl_dict = {
# #     "Place ID":place_id,
#     "Name":name,
#     "Rating":rating,
#     "Date":dates,
#     "User Total Rating":user_ratings_total,
#     "Latitude":lat,
#     "Longatiude":lng,
#     "Business Status":business_status,
#     "Price Level":price_level,


